In [15]:
from urllib2 import urlopen
from json import load
import re
import pandas as pd

In [16]:
transcriptRE = re.compile('storyId=(\d+)')
mp3RE = re.compile('\.mp3\?')

In [17]:
def find_transcript_link(stories):
    if 'transcript' in stories.keys():
        transcript_text = stories['transcript']['link'][0]['$text']
        end_transcript_url = transcriptRE.search(transcript_text).end()
        transcript_link = transcript_text[:end_transcript_url]
        return transcript_link

In [18]:
def find_mp3_link(stories):
    if 'audio' in stories.keys():
        audio = stories['audio'][0]['format']
        if 'mp3' in audio.keys():
            mp3_text = ['mp3'][0]['$text']
            end_mp3_url = mp3RE.search(mp3_text).end() - 1
            mp3_link = mp3_text[:end_mp3_url]
            return mp3_link

In [19]:
def find_byline_name(story):
    if 'byline' in story.keys():
        byline = story['byline'][0]
        if 'name' in byline.keys():
            person_name = byline['name']['$text']

In [20]:
def find_byline_id(story):
    if 'byline' in story.keys():
        byline = story['byline'][0]
        if 'name' in byline.keys():
            if 'personId' in byline['name'].keys():
                person_name = byline['name']['personId']

In [21]:
def find_program_id(story):
    if 'show' in story.keys():
        program = story['show'][0]['program']
        if 'id' in program.keys():
            return program['id']

In [22]:
def find_program_code(story):
    if 'show' in story.keys():
        program = story['show'][0]['program']
        if 'code' in program.keys():
            return program['code']

In [23]:
def find_program_text(story):
    if 'show' in story.keys():
        program = story['show'][0]['program']
        if '$text' in program.keys():
            return program['$text']

In [24]:
# def find_show(story):
#     if 'show' in story.keys():
#         program_id = story['show'][0]['program']['id']
#         program_code = story['show'][0]['program']['code']
#         program_text = story['show'][0]['program']['text']
#         return program_id, program_code, program_text

In [25]:
# base url + the apiKey param
url = 'http://api.npr.org/query?&output=JSON&apiKey='
key = 'MDI2MDExNjc3MDE0NzE4OTMwOTdmNjA1OA000'
fields = '&fields=title,show,byline,text,audio,transcript'
requiredAssets = '&requiredAssets=title,byline,audio,transcript'
url += key + fields + requiredAssets

In [26]:
collect = []

In [27]:
iteration = 0
numResults = 20
returned_story_count = numResults

while returned_story_count == numResults:
    
    # keep track of how far back we go
    startNum = iteration * numResults + 1
    url = url + '&numResults=' + str(numResults) + '&startNum=' + str(startNum)
    
    # open our url, load the JSON
    response = urlopen(url)
    json_obj = load(response)
    returned_story_count = len(json_obj['list']['story'])
    print "RETURNED", returned_story_count
    
    ## CHECK returned_story_count
    
    for story in json_obj['list']['story']:
        story_id = story['id']
        byline_name = find_byline_name(story)
        byline_id = find_byline_id(story)
        program_id = find_program_id(story)
        program_code = find_program_code(story)
        program_text = find_program_text(story)
        title = story['title']['$text']
        transcript = find_transcript_link(story)
        mp3 = find_mp3_link(story)
        
        collect.append((story_id, byline_name, byline_id, program_id, program_code, program_text, title, transcript, mp3))
    
    iteration += 1
    if iteration > 1:
        break

RETURNED 20


TypeError: string indices must be integers, not str

In [ ]:
# story['byline'][0]['name']

In [14]:
print story['byline'][0]['name']['$text']
print ['personId']

Robin Hilton
['personId']


In [26]:
npr = pd.DataFrame.from_records(collect, columns=['story_id', 'byline_name', 'byline_id', 'program_id', 'program_code', 'program_text', 'title', 'transcript', 'mp3'])

In [36]:
npr_complete = npr[npr['transcript'].notnull() & npr['mp3'].notnull()]

In [38]:
# npr_complete['transcript'][0]
npr_complete['mp3'][0]

u'https://ondemand.npr.org/anon.npr-mp3/npr/atc/2016/09/20160905_atc_new_us_open.mp3'

In [35]:
#npr.index = npr['story_id']
print npr.loc['492674834']['transcript']
print npr.loc['492674834']['mp3']

http://www.npr.org/templates/transcript/transcript.php?storyId=492674834
https://ondemand.npr.org/anon.npr-mp3/npr/me/2016/09/20160905_me_german_chancellor_merkels_party_has_a_poor_showing_in_her_home_state_election.mp3


In [40]:
response = urlopen(npr.loc['492674834']['mp3'])
m = response.read()

In [37]:
npr.shape

(40, 7)

In [38]:
npr

,story_id,program_id,program_code,program_text,title,transcript,mp3
0,492305430,None,None,None,The Perils of Power,None,http://play.podtrac.com/npr-510308/ondemand.np...
1,492742613,2,ATC,None,Venus Williams Knocked Out Of U.S. Open After ...,http://www.npr.org/templates/transcript/transc...,https://ondemand.npr.org/anon.npr-mp3/npr/atc/...
2,492742484,2,ATC,None,"Trump, Clinton Mix Things Up On The Campaign T...",http://www.npr.org/templates/transcript/transc...,https://ondemand.npr.org/anon.npr-mp3/npr/atc/...
3,492727633,2,ATC,None,"Truck Drivers Block Highway Near Calais, Franc...",http://www.npr.org/templates/transcript/transc...,https://ondemand.npr.org/anon.npr-mp3/npr/atc/...
4,492727626,2,ATC,None,Hollywood Offers New Batch Of Likely Awards Co...,http://www.npr.org/templates/transcript/transc...,https://ondemand.npr.org/anon.npr-mp3/npr/atc/...
5,492727619,2,ATC,None,U.S. Open Quarterfinals Set To Begin In New York,http://www.npr.org/templates/transcript/transc...,https://ondemand.npr.org/anon.npr-mp3/npr/atc/...
6,492727612,2,ATC,None,Missing Boy's Remains Found Nearly 27 Years Af...,http://www.npr.org/templates/transcript/transc...,https://ondemand.npr.org/anon.npr-mp3/npr/atc/...
7,492727605,2,ATC,None,"Trump, Clinton Make Pitches To Working Class V...",http://www.npr.org/templates/transcript/transc...,https://ondemand.npr.org/anon.npr-mp3/npr/atc/...
8,492727598,2,ATC,None,White House Proposes Loosening Restrictions Fo...,http://www.npr.org/templates/transcript/transc...,https://ondemand.npr.org/anon.npr-mp3/npr/atc/...
9,492727591,2,ATC,None,"Lithuania Welcomes Migrants, But Few Want To Come",http://www.npr.org/templates/transcript/transc...,https://ondemand.npr.org/anon.npr-mp3/npr/atc/...


In [16]:
story.keys()

[u'show', u'text', u'title', u'link', u'audio', u'transcript', u'id']

In [41]:
len(json_obj['list']['story'])

20

In [54]:
stories

{u'audio': [{u'description': {},
   u'duration': {u'$text': u'3866'},
   u'format': {u'hlsOnDemand': {u'$text': u'http://hls.npr.org/nprhls//npr/invsb/2016/07/20160728_invsb_outsidein/master.m3u8'},
    u'mediastream': {u'$text': u'rtmp://flash.npr.org/ondemand/mp3:anon.npr-mp3/npr/invsb/2016/07/20160728_invsb_outsidein.mp3'},
    u'mp3': [{u'$text': u'https://ondemand.npr.org/anon.npr-mp3/npr/invsb/2016/07/20160728_invsb_outsidein.mp3?orgId=1&d=3866&p=64&story=487807747&t=progseg&e=485606589&seg=4&ft=nprml&f=64',
      u'type': u'mp3'},
     {u'$text': u'https://ondemand.npr.org/anon.npr-mp3/npr/invsb/2016/07/20160728_invsb_outsidein.mp3?orgId=1&d=3866&p=64&story=487807747&t=progseg&e=485606589&seg=4&ft=nprml&f=64',
      u'type': u'mp3'},
     {u'$text': u'http://api.npr.org/m3u/1487807795-6bfa73.m3u?orgId=1&d=3866&p=64&story=487807747&t=progseg&e=485606589&seg=4&ft=nprml&f=64',
      u'type': u'm3u'},
     {u'$text': u'http://api.npr.org/m3u/8487807795-fa6ee3.m3u?orgId=1&d=3866&p=64